In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
gpu_information = !nvidia-smi
gpu_information = '\n'.join(gpu_information)
if gpu_information.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator and then re-execute this cell.')
else:
    print(gpu_information)

In [ ]:
from psutil import virtual_memory

ram = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM.'.format(ram))
if ram < 20:
    print('To enable a high-RAM runtime, select the Runtime > "Change runtime type", then select High-RAM in the Runtime shape dropdown '
          'and then re-execute this cell.')
else:
    print('You are using a high-RAM runtime.')

In [ ]:
from tensorflow import config

physical_devices = config.list_physical_devices('GPU')
try:
    config.experimental.set_memory_growth(physical_devices[0], True)
except Exception as exception:
    print(exception)

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

def get_data():
    (train_x, train_y), (test_x, test_y) = cifar10.load_data()
    train_y = to_categorical(train_y)
    test_y = to_categorical(test_y)
    return train_x, train_y, test_x, test_y

In [ ]:
from tensorflow.keras.layers import BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import L2

def get_model(train_x, train_y):
    weight_decay = 1e-4
    model = Sequential(name='model')
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', 
                     activation='elu', kernel_regularizer=L2(l2=weight_decay),
                     input_shape=train_x.shape[1:], name='conv_1'))
    model.add(BatchNormalization(name='batch_norm_1'))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', 
                     activation='elu', kernel_regularizer=L2(l2=weight_decay), 
                     name='conv_2'))
    model.add(BatchNormalization(name='batch_norm_2'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same',
                        name='max_pool_1'))
    model.add(Dropout(rate=0.2, name='dropout_1'))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', 
                    activation='elu', kernel_regularizer=L2(l2=weight_decay), 
                     name='conv_3'))
    model.add(BatchNormalization(name='batch_norm_3'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', 
                     activation='elu', kernel_regularizer=L2(l2=weight_decay), 
                     name='conv_4'))
    model.add(BatchNormalization(name='batch_norm_4'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same',
                        name='max_pool_2'))
    model.add(Dropout(rate=0.3, name='dropout_2'))

    model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same', 
                     activation='elu', kernel_regularizer=L2(l2=weight_decay), 
                     name='conv_5'))
    model.add(BatchNormalization(name='batch_norm_5'))
    model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same', 
                     activation='elu', kernel_regularizer=L2(l2=weight_decay), 
                     name='conv_6'))
    model.add(BatchNormalization(name='batch_norm_6'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same',
                        name='max_pool_3'))
    model.add(Dropout(rate=0.4, name='dropout_3'))

    # model.add(Conv2D(filters=256, kernel_size=(3, 3), padding='same', 
    #                  activation='elu', kernel_regularizer=L2(l2=weight_decay)))
    # model.add(BatchNormalization(name='batch_norm_7'))
    # model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same',
    #                     name='max_pool_4'))
    # model.add(Dropout(rate=0.5, name='dropout_4'))

    model.add(Flatten(name='flatten'))
    model.add(Dense(units=128, activation='elu', name='dense_1'))
    model.add(Dropout(rate=0.5, name='dropout_4'))
    model.add(Dense(units=train_y.shape[1], activation='softmax', name='dense_2'))

    return model

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def get_image_data_generator():
    return ImageDataGenerator(rotation_range=15, width_shift_range=0.1,
                              height_shift_range=0.1, horizontal_flip=True)

In [ ]:
def get_learning_rate(epoch):
    if epoch > 125:
        return 1e-4
    if epoch > 100:
        return 3e-4
    if epoch > 75:
        return 5e-4
    if epoch > 50:
        return 7e-4
    return 1e-3

In [ ]:
train_x, train_y, test_x, test_y = get_data()

In [ ]:
from tensorflow import device
from tensorflow.keras.optimizers import Adam
from keras.utils.vis_utils import plot_model

with device('gpu:0'):
    model = get_model(train_x, train_y)
    model.compile(optimizer=Adam(learning_rate=1e-3), 
                  loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
model_path = '/content/gdrive/My Drive/Master/Anul 1/Retele neuronale/Teme/Tema 4/model.h5'

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint

batch_size = 128
image_data_generator = get_image_data_generator()
image_data_generator.fit(train_x)
with device('gpu:0'):
    model.fit(image_data_generator.flow(x=train_x, y=train_y, 
                                        batch_size=batch_size),
              batch_size=batch_size, epochs=200,
              callbacks=[LearningRateScheduler(get_learning_rate),
                         ModelCheckpoint(model_path, monitor='val_accuracy',
                                         verbose=1, save_best_only=True,
                                         mode='max')],
              validation_data=(test_x, test_y))

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
 
(train_x, train_y), (test_x, test_y) = cifar10.load_data() 
model = load_model(model_path)
model.summary()
scores = model.evaluate(test_x, to_categorical(test_y))
print('Loss: %.3f' % scores[0])
print('Accuracy: %.3f' % scores[1])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_1 (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_norm_1 (BatchNormaliz  (None, 32, 32, 32)       128       
 ation)                                                          
                                                                 
 conv_2 (Conv2D)             (None, 32, 32, 32)        9248      
                                                                 
 batch_norm_2 (BatchNormaliz  (None, 32, 32, 32)       128       
 ation)                                                          
                                                                 
 max_pool_1 (MaxPooling2D)   (None, 16, 16, 32)        0         
                                                                 
 dropout_1 (Dropout)         (None, 16, 16, 32)        0     